In [19]:
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config))
#import keras
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.models import Model
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from matplotlib.pyplot import imshow
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation, Conv2D, MaxPooling2D,BatchNormalization,GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


In [20]:
train=pd.read_csv("/home/sainathb/dog breed files/Image_Classification/labels.csv")

In [21]:
ylabels=train['breed']

In [22]:
ylabels=np.array(pd.get_dummies(ylabels))
ylabels.shape

(10222, 120)

In [23]:
from keras.preprocessing import image
def read_img(img_path):
    img = image.load_img(img_path,target_size=(400,400,3))
    arr=image.img_to_array(img)
    return arr/255

In [24]:
def crop_read_img(img_path):
    img = image.load_img(img_path,target_size=(400,400,3))
    arr=image.img_to_array(img)
    return arr/255

In [25]:
trainimages=[]
for img_path in tqdm(train['id'].values):
        trainimages.append(read_img("/home/sainathb/dog breed files/Image_Classification/train/"+ img_path+".jpg"))

100%|██████████| 10222/10222 [01:04<00:00, 159.56it/s]


In [26]:
trainimages=np.array(trainimages)

In [27]:
croptrainimages=[]
for img_path in tqdm(train['id'].values):
        croptrainimages.append(crop_read_img("/home/sainathb/dog breed files/Image_Classification/traincopy/train/"+ img_path+".jpg"))

100%|██████████| 10222/10222 [00:51<00:00, 199.10it/s]


In [28]:
croptrainimages=np.array(croptrainimages)

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainimages,ylabels, test_size=0.1,shuffle=True)

In [34]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=5)
for train_index, test_index in sss.split(trainimages,ylabels):
        X_train,X_test = trainimages[train_index], trainimages[test_index]
        y_train, y_test = ylabels[train_index], ylabels[test_index]

In [35]:
X_train.shape

(9199, 400, 400, 3)

In [66]:
for train_index, test_index in sss.split(croptrainimages,ylabels):
        cX_train,cX_test = croptrainimages[train_index], croptrainimages[test_index]
        cy_train, cy_test = ylabels[train_index], ylabels[test_index]

In [31]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

base_model1= keras.applications.xception.Xception(include_top=False, weights='imagenet',input_shape=(400,400,3),pooling='avg')

base_model2= keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet',input_shape=(400,400,3),pooling='avg')

base_model3 =InceptionResNetV2(include_top=False , weights='imagenet',input_shape=(400,400,3),pooling='avg')


In [39]:
xpreds=base_model1.predict(x=X_train,batch_size=50)
print(xpreds.shape)
ipreds=base_model2.predict(x=X_train,batch_size=50)
print(ipreds.shape)
irpreds=base_model3.predict(x=X_train,batch_size=50)
print(irpreds.shape)

(9199, 2048)
(9199, 2048)
(9199, 1536)


In [40]:
Xtrain=np.concatenate([irpreds,ipreds,xpreds],axis=-1)
Xtrain.shape

(9199, 5632)

In [41]:
xfpreds=base_model1.predict(x=np.flip(X_train,axis=2),batch_size=10)
print(xfpreds.shape)
ifpreds=base_model2.predict(x=np.flip(X_train,axis=2),batch_size=10)
print(ifpreds.shape)
irfpreds=base_model3.predict(x=np.flip(X_train,axis=2),batch_size=10)
print(irfpreds.shape)

(9199, 2048)
(9199, 2048)
(9199, 1536)


In [42]:
Xftrain=np.concatenate([irfpreds,ifpreds,xfpreds],axis=-1)
Xftrain.shape

(9199, 5632)

In [69]:
cxpreds=base_model1.predict(x=cX_train,batch_size=10)
print(cxpreds.shape)
cipreds=base_model2.predict(x=cX_train,batch_size=10)
print(cipreds.shape)
cirpreds=base_model3.predict(x=cX_train,batch_size=10)
print(cirpreds.shape)

(9199, 2048)
(9199, 2048)
(9199, 1536)


In [70]:
cXtrain=np.concatenate([cirpreds,cipreds,cxpreds],axis=-1)
cXtrain.shape

(9199, 5632)

In [ ]:
cxfpreds=base_model1.predict(x=np.flip(cX_train,axis=2),batch_size=100)
print(cxfpreds.shape)
cifpreds=base_model2.predict(x=np.flip(cX_train,axis=2),batch_size=100)
print(cifpreds.shape)
cirfpreds=base_model3.predict(x=np.flip(cX_train,axis=2),batch_size=100)
print(cirfpreds.shape)

In [ ]:
cXftrain=np.concatenate([cirfpreds,cifpreds,cxfpreds],axis=-1)
cXftrain.shape

In [43]:
txpreds=base_model1.predict(x=X_test,batch_size=30)
tipreds=base_model2.predict(x=X_test,batch_size=30)
tirpreds=base_model3.predict(x=X_test,batch_size=30)

In [44]:
txfpreds=base_model1.predict(x=np.flip(X_test,axis=2),batch_size=50)
tifpreds=base_model2.predict(x=np.flip(X_test,axis=2),batch_size=50)
tirfpreds=base_model3.predict(x=np.flip(X_test,axis=2),batch_size=50)

In [45]:
Xtest=np.concatenate([tirpreds,tipreds,txpreds],axis=-1)
Xtest.shape

(1023, 5632)

In [46]:
Xftest=np.concatenate([tirfpreds,tifpreds,txfpreds],axis=-1)
Xftest.shape

(1023, 5632)

In [71]:
ctxpreds=base_model1.predict(x=cX_test,batch_size=10)
ctipreds=base_model2.predict(x=cX_test,batch_size=100)
ctirpreds=base_model3.predict(x=cX_test,batch_size=100)



In [ ]:
ctxfpreds=base_model1.predict(x=np.flip(cX_test,axis=2),batch_size=50)
ctifpreds=base_model2.predict(x=np.flip(cX_test,axis=2),batch_size=50)
ctirfpreds=base_model3.predict(x=np.flip(cX_test,axis=2),batch_size=50)

In [72]:
cXtest=np.concatenate([ctirpreds,ctipreds,ctxpreds],axis=-1)
cXtest.shape

(1023, 5632)

In [ ]:
cXftest=np.concatenate([ctirfpreds,ctifpreds,ctxfpreds],axis=-1)
cXftest.shape

In [64]:
def mymodel():
    model = Sequential()
    model.add(Dense(2048,input_shape=(5632,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(120))
    model.add(Activation('softmax'))
    return model

In [47]:
callbacks1=keras.callbacks.ModelCheckpoint('moda.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks2=keras.callbacks.ModelCheckpoint('modb.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks3=keras.callbacks.ModelCheckpoint('modc.h5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks4=keras.callbacks.ModelCheckpoint('modd.h5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)

In [85]:
tensor1=keras.callbacks.TensorBoard(log_dir='./logs/model1/', histogram_freq=0, batch_size=120, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

tensor2=keras.callbacks.TensorBoard(log_dir='./logs/model2', histogram_freq=0, batch_size=120, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [86]:
model1=mymodel()
model1.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.SGD(lr=0.085, decay=0.03, momentum=0.92, nesterov=True), metrics=['accuracy'])
model1.fit(Xtrain,y_train, epochs=120,batch_size=120, shuffle=True,verbose=1,validation_data=(Xtest,y_test),callbacks=[callbacks1,tensor1])

Train on 9199 samples, validate on 1023 samples
Epoch 1/120
9199/9199 [==============================] - 2s 167us/step - loss: 1.1239 - acc: 0.7705 - val_loss: 0.2368 - val_acc: 0.9306

Epoch 00001: val_loss did not improve
Epoch 2/120
9199/9199 [==============================] - 1s 99us/step - loss: 0.2163 - acc: 0.9311 - val_loss: 0.1740 - val_acc: 0.9492

Epoch 00002: val_loss did not improve
Epoch 3/120
9199/9199 [==============================] - 1s 101us/step - loss: 0.1750 - acc: 0.9439 - val_loss: 0.1722 - val_acc: 0.9482

Epoch 00003: val_loss did not improve
Epoch 4/120
9199/9199 [==============================] - 1s 98us/step - loss: 0.1612 - acc: 0.9490 - val_loss: 0.1730 - val_acc: 0.9560

Epoch 00004: val_loss did not improve
Epoch 5/120
9199/9199 [==============================] - 1s 98us/step - loss: 0.1497 - acc: 0.9510 - val_loss: 0.1666 - val_acc: 0.9501

Epoch 00005: val_loss did not improve
Epoch 6/120
9199/9199 [==============================] - 1s 98us/step - los

9199/9199 [==============================] - 1s 97us/step - loss: 0.0894 - acc: 0.9745 - val_loss: 0.1604 - val_acc: 0.9501

Epoch 00047: val_loss did not improve
Epoch 48/120
9199/9199 [==============================] - 1s 100us/step - loss: 0.0910 - acc: 0.9711 - val_loss: 0.1601 - val_acc: 0.9521

Epoch 00048: val_loss did not improve
Epoch 49/120
9199/9199 [==============================] - 1s 100us/step - loss: 0.0897 - acc: 0.9724 - val_loss: 0.1605 - val_acc: 0.9501

Epoch 00049: val_loss did not improve
Epoch 50/120
9199/9199 [==============================] - 1s 98us/step - loss: 0.0892 - acc: 0.9748 - val_loss: 0.1602 - val_acc: 0.9511

Epoch 00050: val_loss did not improve
Epoch 51/120
9199/9199 [==============================] - 1s 98us/step - loss: 0.0892 - acc: 0.9731 - val_loss: 0.1603 - val_acc: 0.9511

Epoch 00051: val_loss did not improve
Epoch 52/120
9199/9199 [==============================] - 1s 98us/step - loss: 0.0884 - acc: 0.9745 - val_loss: 0.1600 - val_acc: 0


Epoch 00093: val_loss did not improve
Epoch 94/120
9199/9199 [==============================] - 1s 97us/step - loss: 0.0801 - acc: 0.9775 - val_loss: 0.1600 - val_acc: 0.9501

Epoch 00094: val_loss did not improve
Epoch 95/120
9199/9199 [==============================] - 1s 100us/step - loss: 0.0811 - acc: 0.9775 - val_loss: 0.1599 - val_acc: 0.9511

Epoch 00095: val_loss did not improve
Epoch 96/120
9199/9199 [==============================] - 1s 100us/step - loss: 0.0804 - acc: 0.9771 - val_loss: 0.1597 - val_acc: 0.9501

Epoch 00096: val_loss did not improve
Epoch 97/120
9199/9199 [==============================] - 1s 98us/step - loss: 0.0780 - acc: 0.9788 - val_loss: 0.1598 - val_acc: 0.9511

Epoch 00097: val_loss did not improve
Epoch 98/120
9199/9199 [==============================] - 1s 97us/step - loss: 0.0817 - acc: 0.9752 - val_loss: 0.1597 - val_acc: 0.9501

Epoch 00098: val_loss did not improve
Epoch 99/120
9199/9199 [==============================] - 1s 98us/step - loss: 

In [87]:
model2=mymodel()
model2.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.SGD(lr=0.085, decay=0.03, momentum=0.92, nesterov=True), metrics=['accuracy'])
model2.fit(Xftrain,y_train, epochs=150,batch_size=120, shuffle=True,verbose=1,validation_data=(Xftest,y_test),callbacks=[callbacks2,tensor2])
   

Train on 9199 samples, validate on 1023 samples
Epoch 1/150
9199/9199 [==============================] - 2s 167us/step - loss: 1.1236 - acc: 0.7638 - val_loss: 0.2310 - val_acc: 0.9267

Epoch 00001: val_loss did not improve
Epoch 2/150
9199/9199 [==============================] - 1s 99us/step - loss: 0.2222 - acc: 0.9292 - val_loss: 0.1823 - val_acc: 0.9443

Epoch 00002: val_loss did not improve
Epoch 3/150
9199/9199 [==============================] - 1s 98us/step - loss: 0.1803 - acc: 0.9401 - val_loss: 0.1640 - val_acc: 0.9492

Epoch 00003: val_loss did not improve
Epoch 4/150
9199/9199 [==============================] - 1s 97us/step - loss: 0.1598 - acc: 0.9495 - val_loss: 0.1656 - val_acc: 0.9453

Epoch 00004: val_loss did not improve
Epoch 5/150
9199/9199 [==============================] - 1s 101us/step - loss: 0.1488 - acc: 0.9524 - val_loss: 0.1639 - val_acc: 0.9482

Epoch 00005: val_loss did not improve
Epoch 6/150
9199/9199 [==============================] - 1s 100us/step - lo

9199/9199 [==============================] - 1s 99us/step - loss: 0.0925 - acc: 0.9727 - val_loss: 0.1574 - val_acc: 0.9492

Epoch 00047: val_loss did not improve
Epoch 48/150
9199/9199 [==============================] - 1s 98us/step - loss: 0.0888 - acc: 0.9723 - val_loss: 0.1573 - val_acc: 0.9492

Epoch 00048: val_loss did not improve
Epoch 49/150
9199/9199 [==============================] - 1s 98us/step - loss: 0.0916 - acc: 0.9729 - val_loss: 0.1569 - val_acc: 0.9482

Epoch 00049: val_loss did not improve
Epoch 50/150
9199/9199 [==============================] - 1s 100us/step - loss: 0.0925 - acc: 0.9743 - val_loss: 0.1569 - val_acc: 0.9472

Epoch 00050: val_loss did not improve
Epoch 51/150
9199/9199 [==============================] - 1s 99us/step - loss: 0.0920 - acc: 0.9725 - val_loss: 0.1574 - val_acc: 0.9482

Epoch 00051: val_loss did not improve
Epoch 52/150
9199/9199 [==============================] - 1s 98us/step - loss: 0.0898 - acc: 0.9737 - val_loss: 0.1572 - val_acc: 0.


Epoch 00093: val_loss did not improve
Epoch 94/150
9199/9199 [==============================] - 1s 97us/step - loss: 0.0839 - acc: 0.9754 - val_loss: 0.1574 - val_acc: 0.9482

Epoch 00094: val_loss did not improve
Epoch 95/150
9199/9199 [==============================] - 1s 99us/step - loss: 0.0837 - acc: 0.9748 - val_loss: 0.1576 - val_acc: 0.9482

Epoch 00095: val_loss did not improve
Epoch 96/150
9199/9199 [==============================] - 1s 99us/step - loss: 0.0847 - acc: 0.9745 - val_loss: 0.1572 - val_acc: 0.9482

Epoch 00096: val_loss did not improve
Epoch 97/150
9199/9199 [==============================] - 1s 98us/step - loss: 0.0841 - acc: 0.9756 - val_loss: 0.1577 - val_acc: 0.9492

Epoch 00097: val_loss did not improve
Epoch 98/150
9199/9199 [==============================] - 1s 100us/step - loss: 0.0839 - acc: 0.9753 - val_loss: 0.1572 - val_acc: 0.9492

Epoch 00098: val_loss did not improve
Epoch 99/150
9199/9199 [==============================] - 1s 98us/step - loss: 0

9199/9199 [==============================] - 1s 98us/step - loss: 0.0770 - acc: 0.9778 - val_loss: 0.1575 - val_acc: 0.9492

Epoch 00140: val_loss did not improve
Epoch 141/150
9199/9199 [==============================] - 1s 100us/step - loss: 0.0791 - acc: 0.9778 - val_loss: 0.1573 - val_acc: 0.9482

Epoch 00141: val_loss did not improve
Epoch 142/150
9199/9199 [==============================] - 1s 98us/step - loss: 0.0775 - acc: 0.9778 - val_loss: 0.1576 - val_acc: 0.9492

Epoch 00142: val_loss did not improve
Epoch 143/150
9199/9199 [==============================] - 1s 98us/step - loss: 0.0756 - acc: 0.9786 - val_loss: 0.1574 - val_acc: 0.9482

Epoch 00143: val_loss did not improve
Epoch 144/150
9199/9199 [==============================] - 1s 99us/step - loss: 0.0770 - acc: 0.9777 - val_loss: 0.1574 - val_acc: 0.9482

Epoch 00144: val_loss did not improve
Epoch 145/150
9199/9199 [==============================] - 1s 99us/step - loss: 0.0768 - acc: 0.9776 - val_loss: 0.1576 - val_ac

In [74]:
model3=mymodel()
model3.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.SGD(lr=0.085, decay=0.03, momentum=0.92, nesterov=True), metrics=['accuracy'])
model3.fit(cXtrain,cy_train, epochs=100,batch_size=120, shuffle=True,verbose=1,validation_data=(cXtest,cy_test),callbacks=[callbacks3])

Train on 9199 samples, validate on 1023 samples
Epoch 1/100
9199/9199 [==============================] - 7s 747us/step - loss: 1.1974 - acc: 0.7445 - val_loss: 0.3321 - val_acc: 0.9042

Epoch 00001: val_acc improved from -inf to 0.90420, saving model to modc.h5
Epoch 2/100
9199/9199 [==============================] - 1s 98us/step - loss: 0.3219 - acc: 0.9041 - val_loss: 0.2844 - val_acc: 0.9198

Epoch 00002: val_acc improved from 0.90420 to 0.91984, saving model to modc.h5
Epoch 3/100
9199/9199 [==============================] - 1s 99us/step - loss: 0.2532 - acc: 0.9251 - val_loss: 0.2657 - val_acc: 0.9277

Epoch 00003: val_acc improved from 0.91984 to 0.92766, saving model to modc.h5
Epoch 4/100
9199/9199 [==============================] - 1s 99us/step - loss: 0.2302 - acc: 0.9327 - val_loss: 0.2669 - val_acc: 0.9238

Epoch 00004: val_acc did not improve
Epoch 5/100
9199/9199 [==============================] - 1s 103us/step - loss: 0.2153 - acc: 0.9360 - val_loss: 0.2610 - val_acc: 0.

KeyboardInterrupt: 

In [ ]:
model4=mymodel()
model4.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model4.fit(cXftrain,cy_train, epochs=100,batch_size=150, shuffle=True,verbose=1,validation_data=(cXftest,cy_test))

In [8]:
from tqdm import tqdm
import cv2
test=pd.read_csv("/home/sainathb/dog breed files/Image_Classification/sample_submission.csv")
xtest=[]
for img_path in tqdm(test['id'].values):
        xtest.append(read_img("/home/sainathb/dog breed files/Image_Classification/test/"+ img_path+".jpg"))
xtest=np.array(xtest)

100%|██████████| 10357/10357 [01:07<00:00, 153.01it/s]


In [75]:
cxtest=[]
for img_path in tqdm(test['id'].values):
        cxtest.append(crop_read_img("/home/sainathb/dog breed files/Image_Classification/testcopy/"+ img_path+".jpg"))
cxtest=np.array(cxtest)

 76%|███████▌  | 7862/10357 [00:43<00:13, 178.91it/s]

KeyboardInterrupt: 

In [76]:
model1.load_weights('./moda.h5')
model2.load_weights('./modb.h5')

 76%|███████▌  | 7862/10357 [01:00<00:19, 130.96it/s]

In [77]:
pxpreds=base_model1.predict(x=xtest,batch_size=10)
pipreds=base_model2.predict(x=xtest,batch_size=10)
pirpreds=base_model3.predict(x=xtest,batch_size=10)


In [78]:
pXtest=np.concatenate([pirpreds,pipreds,pxpreds],axis=-1)
pXtest.shape

(10357, 5632)

In [79]:
pxfpreds=base_model1.predict(x=np.flip(xtest,axis=2),batch_size=50)
pifpreds=base_model2.predict(x=np.flip(xtest,axis=2),batch_size=50)
pirfpreds=base_model3.predict(x=np.flip(xtest,axis=2),batch_size=50)

In [80]:
pXftest=np.concatenate([pirfpreds,pifpreds,pxfpreds],axis=-1)
pXftest.shape

(10357, 5632)

In [ ]:
cpxpreds=base_model1.predict(x=cxtest,batch_size=100)
cpipreds=base_model2.predict(x=cxtest,batch_size=100)
cpirpreds=base_model3.predict(x=cxtest,batch_size=100)

In [ ]:
cpXtest=np.concatenate([cpirpreds,cpipreds,cpxpreds],axis=-1)
cpXtest.shape

In [ ]:
cpxfpreds=base_model1.predict(x=np.flip(cxtest,axis=2),batch_size=50)
cpifpreds=base_model2.predict(x=np.flip(cxtest,axis=2),batch_size=50)
cpirfpreds=base_model3.predict(x=np.flip(cxtest,axis=2),batch_size=50)

In [ ]:
cpXftest=np.concatenate([cpirfpreds,cpifpreds,cpxfpreds],axis=-1)
cpXftest.shape

In [ ]:
model1.load_weights('model11.h5')
model2.load_weights('model21.h5')
model3.load_weights('model31.h5')
model4.load_weights('model41.h5')

In [81]:
a=model1.predict(pXtest)
b=model2.predict(pXftest)

In [82]:
e=(a+b)/2

In [83]:
e.shape

(10357, 120)

In [84]:
np.savetxt('zeropoint15.csv',e, delimiter=',',fmt="%f")

In [ ]:
model1.save_weights('normal.h5')
model2.save_weights('normalf.h5')
model3.save_weights('normalc.h5')
model4.save_weights('normalcf.h5')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
plt.style.use('ggplot')

%matplotlib inline

In [ ]:
preds=np.argmax(c,1)

In [ ]:
y=np.array(y_test)

In [ ]:
y=np.argmax(y,1)

In [ ]:
breeds=np.unique(train['breed'].values)

In [ ]:
pr=[]
for i in preds:
    pr.append(breeds[i])

In [ ]:
pr1=[]
for i in y:
    pr1.append(breeds[i])

In [ ]:
cm=confusion_matrix(pr1,pr,labels=breeds)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 18))
_ = sns.heatmap(cm, ax=ax, yticklabels=breeds, xticklabels=breeds, robust=True)

In [ ]:
pr1=np.array(pr1)
pr1.shape
pr1=pr1.reshape(2045,1)

In [ ]:
pr=np.array(pr)
pr=pr.reshape(2045,1)

In [ ]:
x=np.concatenate((pr1,pr),axis=1)

In [ ]:
x.shape

In [ ]:
np.savetxt("somefile.csv",x,fmt="%s",header="actual,pred,count",delimiter=',',comments='')

In [ ]:
df=pd.read_csv("somefile.csv")

In [ ]:
df['count'] = 1

In [ ]:

misclass_df = df[df['actual'] != df['pred']].groupby(['actual', 'pred']).sum().sort_values(['count'], ascending=False).reset_index()
misclass_df['pair'] = misclass_df['actual'] + ' / ' + misclass_df['pred']
misclass_df = misclass_df[['pair', 'count']].take(range(30))

misclass_df.sort_values(['count']).plot.barh(figsize=(8, 10), x=misclass_df['pair'])



In [ ]:
np.savetxt('irix.csv', p, delimiter=',',fmt="%f") 

In [ ]:
from tqdm import tqdm
import cv2
test=pd.read_csv("/home/sainathb/Image_Classification/sample_submission.csv")
xtest=[]
for img_path in tqdm(test['id'].values):
        xtest.append(read_img("/home/sainathb/Image_Classification/test/"+ img_path+".jpg"))
xtest=np.array(xtest)

In [ ]:
base_model4=keras.applications.nasnet.NASNetLarge(input_shape=None, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling='avg')